In [71]:
from seleniumbase import Driver
from selenium.webdriver.common.by import By
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException

In [72]:
driver = Driver(uc=True)   # undetected-chromedriver

In [11]:
BASE_URL = 'https://www.noon.com/egypt-ar/search/?page={}&q=%D8%A7%D9%84%D8%A7%D8%AF%D9%88%D8%A7%D8%AA+%D8%A7%D9%84%D8%B5%D8%AD%D9%8A%D9%87+%D9%84%D9%84%D8%AD%D9%85%D8%A7%D9%85+%D8%AE%D9%84%D8%A7%D8%B7%D8%A7%D8%AA&utm_source=C1000088L&utm_medium=cpc&utm_campaign=C1000151355N_eg_en_web_searchxxexactandphrasexxbrandpurexx08082022_noon_web_c1000088l_acquisition_sembranded'

In [19]:
all_product_links = []

In [20]:
for i in range(1,4):
    time.sleep(3) 
    driver.get(BASE_URL.format(i))
    PRODUCT_NAME_LOCATOR = (By.XPATH, '//h2[@data-qa="plp-product-box-name"]')
    product_name_elements = driver.find_elements(*PRODUCT_NAME_LOCATOR)
    
    if not product_name_elements:
        print("No product elements found on the page.")
    
    for index, name_element in enumerate(product_name_elements):
        link_element = name_element.find_element(By.XPATH, './ancestor::a[1]')
        product_link = link_element.get_attribute('href')
        all_product_links.append(product_link)


In [21]:
all_product_links

['https://www.noon.com/egypt-ar/bathtube-mixer-8587/Z5E3C18E0CB1B90CA833EZ/p/?o=z5e3c18e0cb1b90ca833ez-1',
 'https://www.noon.com/egypt-ar/galaxy-mixer-tap-4-in-1-360-rotation-and-multiple-water-pressures/Z5829D7E9022F95469CEFZ/p/?o=bfad8578aadbc49b',
 'https://www.noon.com/egypt-ar/bath-wash-mixer/Z006AD51477B2F3D80FABZ/p/?o=z006ad51477b2f3d80fabz-1',
 'https://www.noon.com/egypt-ar/kitchen-wall-mount-mixer-with-handles-chrome-bc178aa/Z04E7C91A5430F7548875Z/p/?o=z04e7c91a5430f7548875z-1',
 'https://www.noon.com/egypt-ar/sink-mixer-8577/Z9218AEC9DF442821E16CZ/p/?o=z9218aec9df442821e16cz-1',
 'https://www.noon.com/egypt-ar/720-degree-swivel-sink-faucet-aerator-silver-9x2-5x2-5cm/N51294095A/p/?o=caeaf887aaff8b3d',
 'https://www.noon.com/egypt-ar/kitchen-wall-mount-mixer-with-handles-chrome-561/Z8BB5DA91FEA6C40C8933Z/p/?o=z8bb5da91fea6c40c8933z-1',
 'https://www.noon.com/egypt-ar/movable-faucet-connector-with-internal-filter/Z442D79388C51E62EEAD9Z/p/?o=z442d79388c51e62eead9z-1',
 'https:/

In [112]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

def extract_noon_reviews(driver, url, timeout=15):
    """
    Extract all reviews from a Noon product page.
    
    Args:
        driver: Selenium WebDriver
        url: Product URL
        timeout: Max seconds to wait for reviews to load

    Returns:
        List of dictionaries with review info
    """
    driver.get(url)
    all_reviews = []
    wait = WebDriverWait(driver, timeout)
    product_name = driver.find_element(By.CSS_SELECTOR, "[class*='ProductTitle']").text

    # ---- Gradual scroll until reviews section is visible ----
    def scroll_to_reviews():
        for _ in range(5):
            try:
                review_section = driver.find_element(By.XPATH, "//div[contains(@class,'ProductReviewsFilters-module') and contains(., 'المراجعات')]")
                if review_section.is_displayed():
                    driver.execute_script("arguments[0].scrollIntoView(true);", review_section)
                    return True
            except:
                pass
            driver.execute_script("window.scrollBy(0, 1000);")
            time.sleep(1)
        return False

    if not scroll_to_reviews():
        print("⚠️ Reviews section not found on page")
        return []

    time.sleep(1)  # small wait for rendering

    # ---- Click "View All Reviews" if exists ----
    try:
        view_all = driver.find_element(By.CSS_SELECTOR, ".ViewAllReviewsBtn-module-scss-module__viewMore")
        driver.execute_script("arguments[0].scrollIntoView(true);", view_all)
        time.sleep(0.5)
        view_all.click()
        time.sleep(2)  # wait for all reviews page to load
    except NoSuchElementException:
        pass  # no "View All Reviews" button

    # ---- Wait until at least one review is visible or timeout ----
    try:
        wait.until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, ".ReviewListSection-module-scss-module__JSH-ya__noonReviewItem")
            )
        )
    except TimeoutException:
        print("⚠️ No reviews found for this product")
        return []

    # ---- Extract reviews ----
    review_containers = driver.find_elements(
        By.CSS_SELECTOR, ".ReviewListSection-module-scss-module__JSH-ya__noonReviewItem"
    )

    for r in review_containers:
        try:
            username = r.find_element(By.CSS_SELECTOR, ".ReviewItemHeader-module-scss-module__W2V5NW__userName").text
        except:
            username = "anonymous"

        try:
            date = r.find_element(By.CSS_SELECTOR, ".ReviewItemHeader-module-scss-module__W2V5NW__ratedDate").text
        except:
            date = ""

        try:
            title = r.find_element(By.CSS_SELECTOR, ".TranslatedReviewTitle-module-scss-module__QbaJfW__reviewTitle").text
        except:
            title = ""

        try:
            body = r.find_element(By.CSS_SELECTOR, ".ReviewDescription-module-scss-module__AkZflW__reviewDesc").text
        except:
            body = ""

        try:
            rating_imgs = r.find_elements(By.CSS_SELECTOR, ".RatingStars-module-scss-module__fF13gq__container img")
            rating = sum(1 for img in rating_imgs if "rating" in img.get_attribute("color") and "grey" not in img.get_attribute("color"))
        except:
            rating = None

        all_reviews.append({
            "product": product_name,
            "username": username,
            "date": date,
            "title": title,
            "text": body,
            "rating": rating
        })

    print(f"✅ Found {len(all_reviews)} review(s)")
    return all_reviews


In [113]:
url = "https://www.noon.com/egypt-ar/loq-gaming-laptop-with-15-6-inch-full-hd-1920x1080-ips-display-core-i5-12450hx-processor-16gb-ddr5-ram-512gb-ssd-4gb-nvidia-geforce-rtx-2050-graphics-dos-without-windows-english-arabic-grey/N70148823V/p/?utm_source=C1000088L&utm_medium=cpc&utm_campaign=C1000151355N_eg_en_web_searchxxexactandphrasexxbrandpurexx08082022_noon_web_c1000088l_acquisition_sembranded&o=caaa680fb6a8d82a&shareId=5ea23957-22eb-40d3-8a33-91ee8940ab29"
data = extract_noon_reviews(driver,url)

✅ Found 3 review(s)


In [111]:
data

[{'product': 'لاب توب العاب LOQ بـ 15.6-بوصة الدقة العالية (1920x1080) اي بي اس شاشة, نواة i5-12450HX معالج / 16 جيجابايت دي دي آر ذاكرة عشوائية / 512 جيجابايت وحدة تخزين صلبة / 4 جيجابايت نيفيديا جيفورس ار تي اكس 2050 رسوميات / دوس بدون ويندوز / الإنجليزية/العربية',
  'username': 'Anonymous',
  'date': 'نوفمبر 11, 2025',
  'title': 'طلع Essential',
  'text': 'كنت قلقان ليطلع essential وانهم نسيوا يكتبوها في الوصف وقد كان طلبت اللاب حتى مش مكتوب على الكرتونة كلمة essential بس مواصفاته الداخلية زي دعمه لسوكيت رامة واحدة\n ...المزيد',
  'rating': 1},
 {'product': 'لاب توب العاب LOQ بـ 15.6-بوصة الدقة العالية (1920x1080) اي بي اس شاشة, نواة i5-12450HX معالج / 16 جيجابايت دي دي آر ذاكرة عشوائية / 512 جيجابايت وحدة تخزين صلبة / 4 جيجابايت نيفيديا جيفورس ار تي اكس 2050 رسوميات / دوس بدون ويندوز / الإنجليزية/العربية',
  'username': 'Mahmoud M.',
  'date': 'مارس 25, 2025',
  'title': 'بطاريه اسوء ما يمكن',
  'text': 'الجهاز ك اداء عادي ساعات بيكون في تهنيجبس مقبول بس ك بطاريه اسوء مايمكن البطر

In [ ]:
all_reviews = []
for url in all_product_links:
    reviews = extract_noon_reviews(driver,url)
    all_reviews.extend(reviews)  # Combine all reviews in one list

# Convert to DataFrame
df_all_reviews = pd.DataFrame(all_reviews)


⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
✅ Found 1 review(s)
⚠️ Reviews section not found on page
✅ Found 3 review(s)
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page
⚠️ Reviews section not found on page


In [97]:
df_all_reviews.head()

,product_name,username,date,title,body,rating
0,خلاط مطبخ سيرالوك BC178AA,Hida H.,"يوليو 26, 2025",حقيقي رائع وشكرا لاديال استاندر كان في مشكله ف...,حقيقي رائع,5
1,جهاز تهوية صنبور حوض دوار بزاوية 720 درجة فضي ...,Anonymous,"مايو 12, 2025",مقبول,التركيب صعب شويه \nلكنه في المجمل كويس بيبعد ا...,3
2,جهاز تهوية صنبور حوض دوار بزاوية 720 درجة فضي ...,Ghada I.,"ديسمبر 14, 2023",خامته بلاستيك لكنها تقضي الغرض,خامته بلاستيك لكنها تقضي الغرض سعرها مبالغ فيه...,4
3,جهاز تهوية صنبور حوض دوار بزاوية 720 درجة فضي ...,Anonymous,"ديسمبر 15, 2024",تنفع حوض الحمام فقط,جميله تنفع حوض الحمام فقط,3


In [98]:
df_all_reviews.duplicated().sum()

np.int64(0)

In [99]:
df_all_reviews.drop_duplicates(inplace=True)

In [100]:
df_all_reviews.shape

(4, 6)

In [ ]:
df_all_reviews.to_csv("all_noon_reviews.csv", index=False)